## Dataset Creation

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import librosa

import tensorflow as tf
import pandas as pd
import keras

import os
import yaml

import skimage.io
from tqdm import tqdm


In [3]:
# Example encoding of a label [1, 1, 0, 1, 0, 0]

irmas_labels = ["cel", "cla", "flu", "gac", "sax", "tru", "vio"]

# Define the labels 
# cello, clarinet, flute, acoustic guitar, electric guitar, organ, piano, saxophone, trumpet, violin, and human singing voice
label_names = ["cello", "clarinet", "flute", "guitar", "saxophone", "trumpet", "violin"]
n_classes = len(label_names)


In [7]:
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def save_spectrogram_image(spectrogram, path):
    # min-max scale to fit inside 8-bit range
    img = scale_minmax(spectrogram, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy
    # save as PNG
    skimage.io.imsave(path, img)


from tqdm import tqdm

def wavs_to_imgs(path):
    new_path = f"datasets/IRMAS-TrainingData-images"

    data = []
    for label in tqdm(irmas_labels, desc="Processing Dataset"):
        label_path = f"{path}/{label}"
        for filename in os.listdir(label_path):
            if filename.endswith(".wav"):
                file_path = f"{label_path}/{filename}"
                audio, sr = librosa.load(file_path, sr=None)
                D = librosa.amplitude_to_db(np.abs(librosa.stft(y = audio)), ref=np.max)
                save_spectrogram_image(D, f"{new_path}/{label}/{filename}.png")
                
wavs_to_imgs("datasets/IRMAS-TrainingData")

Processing Dataset: 100%|██████████| 7/7 [01:08<00:00,  9.85s/it]


In [5]:
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def save_spectrogram_image(spectrogram, path):
    # min-max scale to fit inside 8-bit range
    img = scale_minmax(spectrogram, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy
    # save as PNG
    skimage.io.imsave(path, img)


from tqdm import tqdm

def wavs_to_imgs(path):
    new_path = f"datasets/handmade-images"

    data = []
    for label in tqdm(irmas_labels, desc="Processing Dataset"):
        label_path = f"{path}/{label}"
        for filename in os.listdir(label_path):
            if filename.endswith(".wav") or filename.endswith(".aiff") or filename.endswith(".mp3"):
                file_path = f"{label_path}/{filename}"
                audio, sr = librosa.load(file_path, sr=None)
                D = librosa.amplitude_to_db(np.abs(librosa.stft(y = audio[:sr*3])), ref=np.max)
                save_spectrogram_image(D, f"{new_path}/{label}/{filename}.png")
                
wavs_to_imgs("datasets/handmade")

Processing Dataset: 100%|██████████| 7/7 [00:00<00:00, 15.50it/s]


### Data preparation

1. Define classes  
2. Read matadata  
3. Cut each track in 1s long excerpts 
4. Create labels for each 1s long excerpt 
    - For each stem cut in 1s excerpt
    - check if RMS > threshold (0.05?) 
    - set label of that instrument to 1 

In [4]:
# Example encoding of a label [1, 1, 0, 1, 0, 0]

slakh_mapping_labels = {
    "Bass": "bass",
    "Chromatic Percussion": "drums",
    "Drums": "drums",
    "Percussive": "drums",
    "Guitar": "guitar",
    "Piano": "piano",
    "Organ": "other",
    "Strings": "other",
    "Strings (continued)": "other",
    "Brass": "other",
    "Reed": "other",
    "Pipe": "other",
    "Synth Lead": "other",
    "Synth Pad": "other",
    "Sound Effects": "other",
    "Ethnic": "other",
    "Sound effects": "other",
}

def map_slakh_labels(label):
    return slakh_mapping_labels[label]

def label_to_index(label):
    return label_names.index(label)

label_names = ["vocal", "guitar", "bass", "drums", "piano","other"]
n_labels = len(label_names)


In [5]:
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def save_spectrogram_image(spectrogram, path):
    # min-max scale to fit inside 8-bit range
    img = scale_minmax(spectrogram, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy

    # save as PNG
    skimage.io.imsave(path, img)

In [6]:
def read_metadata(metadata_path):
    with open(metadata_path, 'r') as file:
        return yaml.safe_load(file)
    return null

def is_present(signal):
    energy_presence_threshold = 0.1
    return np.any(np.abs(signal) > energy_presence_threshold)  # TODO: Find a better threshold

def create_label(metadata, path, name, index, excerpt_length):
    # print(f"Creating labels for {name}")
    label = np.zeros(n_labels)
    
    for stem in metadata["stems"]:
        try:
            stem_audio, _ = librosa.load(f"{path}/stems/{stem}.wav", sr=None)
            stem_excerpt = stem_audio[index:index+excerpt_length]

            instrument = metadata["stems"][stem]["inst_class"]
            
            label_name = map_slakh_labels(instrument)
            if is_present(stem_excerpt):
                label[label_to_index(label_name)] = 1
        except:
            print(f"Error reading {stem}")
            continue
        
    return label


def create_spectogram(signal, sr, output_name):
    # print(f"Creating spectograms for {output_name}")
    spectogram_db = feature_extraction(signal, sr)
    full_path =  f"datasets/custom/{output_name}.png"
    save_spectrogram_image(spectogram_db, full_path)
    return full_path
    

def process_mix(df, metadata, mix, sr, path, name, excerpt_length_sec = 5):
    excerpt_length = sr * excerpt_length_sec
    for index in range(0, len(mix), excerpt_length):
        if index + excerpt_length > len(mix):
            break

        excerpt = mix[index:index+excerpt_length]
        output_name = f"{name}_{index//sr}"

        png_path = create_spectogram(excerpt, sr, output_name)
        label = create_label(metadata, path, output_name, index, excerpt_length)

        # add row to dataframe with png path and label
        df = df._append({'png_path': png_path, 'label': label}, ignore_index=True)

    return df
    
    # save dataframe to csv

def feature_extraction(signal, sr):
    S = librosa.feature.melspectrogram(y=signal, sr=sr)
    S_dB = librosa.power_to_db(S, ref=np.max)
    return S_dB

In [7]:
directory = 'datasets/babyslakh_16k/'
subdirs = [x[1] for x in os.walk(directory)][0]

# For each subdir: load mix.wav, metadata.yaml 
df = pd.DataFrame(columns=['png_path', 'label'])

for subdir in subdirs:
    path = directory + subdir
    print("Processing: " + path)
    

    mix, sr = librosa.load(path + '/mix.wav', sr=None) # Load the mix
    metadata = read_metadata(path + '/metadata.yaml') # read yaml file

    df = process_mix(df, metadata, mix, sr, path=path, name=subdir)

df.to_csv(f"datasets/custom/data.csv", index=False)

Processing: datasets/babyslakh_16k/Track00006
Processing: datasets/babyslakh_16k/Track00001
Error reading S06


/var/folders/9b/646lmqp56b53pz1fqqr9x5gw0000gn/T/ipykernel_85422/370983470.py:16: UserWarning: PySoundFile failed. Trying audioread instead.
  stem_audio, _ = librosa.load(f"{path}/stems/{stem}.wav", sr=None)
/Users/mariapolak/miniconda3/envs/mlme/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/var/folders/9b/646lmqp56b53pz1fqqr9x5gw0000gn/T/ipykernel_85422/370983470.py:16: UserWarning: PySoundFile failed. Trying audioread instead.
  stem_audio, _ = librosa.load(f"{path}/stems/{stem}.wav", sr=None)
/Users/mariapolak/miniconda3/envs/mlme/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, durat

Error reading S06
Error reading S06


/var/folders/9b/646lmqp56b53pz1fqqr9x5gw0000gn/T/ipykernel_85422/370983470.py:16: UserWarning: PySoundFile failed. Trying audioread instead.
  stem_audio, _ = librosa.load(f"{path}/stems/{stem}.wav", sr=None)
/Users/mariapolak/miniconda3/envs/mlme/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/var/folders/9b/646lmqp56b53pz1fqqr9x5gw0000gn/T/ipykernel_85422/370983470.py:16: UserWarning: PySoundFile failed. Trying audioread instead.
  stem_audio, _ = librosa.load(f"{path}/stems/{stem}.wav", sr=None)
/Users/mariapolak/miniconda3/envs/mlme/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, durat

Error reading S06
Error reading S06


...

Error reading S12
Error reading S13


/var/folders/9b/646lmqp56b53pz1fqqr9x5gw0000gn/T/ipykernel_85422/370983470.py:16: UserWarning: PySoundFile failed. Trying audioread instead.
  stem_audio, _ = librosa.load(f"{path}/stems/{stem}.wav", sr=None)
/Users/mariapolak/miniconda3/envs/mlme/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error reading S12
Error reading S13


/var/folders/9b/646lmqp56b53pz1fqqr9x5gw0000gn/T/ipykernel_85422/370983470.py:16: UserWarning: PySoundFile failed. Trying audioread instead.
  stem_audio, _ = librosa.load(f"{path}/stems/{stem}.wav", sr=None)
/Users/mariapolak/miniconda3/envs/mlme/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error reading S12
Error reading S13
Processing: datasets/babyslakh_16k/Track00020


In [ ]:
    
# Load the one-hot encoding and the mix
# Create a model that takes the mix as input and outputs the one-hot encoding
# Train the model
# Save the model
# Load the model
# Test the model
# Save the test results
# Load the test results
# Plot

In [ ]:
import pandas as pd

# Load labels from a CSV file
labels_df = pd.read_csv('path/to/your/labels.csv')

# Example structure of labels_df
# | filename         | guitar | piano | vocals | drums | violin |
# |------------------|--------|-------|--------|-------|--------|
# | file1.wav        | 1      | 0     | 1      | 0     | 0      |
# | file2.wav        | 0      | 1     | 0      | 1     | 0      |


Code from copilot

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa

# Function to load an audio file and convert it to a spectrogram
def load_audio_file(file_path):
    y, sr = librosa.load(file_path, sr=None)
    S = librosa.feature.melspectrogram(y, sr=sr)
    S_dB = librosa.power_to_db(S, ref=np.max)
    return S_dB

# Load labels from a CSV file
labels_df = pd.read_csv('path/to/your/labels.csv')

# Directory containing audio files
audio_dir = 'path/to/your/audio/files'

# Initialize lists to hold data and labels
data = []
labels = []

# Iterate over each row in the labels DataFrame
for index, row in labels_df.iterrows():
    file_path = os.path.join(audio_dir, row['filename'])
    spectrogram = load_audio_file(file_path)
    data.append(spectrogram)
    labels.append(row[1:].values)

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Reshape data to add a channel dimension
data = data[..., np.newaxis]

# Example shapes
print(data.shape)  # (num_samples, height, width, 1)
print(labels.shape)  # (num_samples, num_classes)
